In [ ]:
# -----------------------
# app.ipynb / app.py
# -----------------------
import os
import dash
import pandas as pd
from dash import dcc, html, Input, Output, dash_table
from strava_utils import load_data, create_folium_map, compute_metrics, create_heatmap_figure, weekly_distance_figure, personal_bests_table


# -----------------------
# Load data
# -----------------------
df = load_data('data/nicole_strava.csv')

# -----------------------
# Initialize Dash
# -----------------------
app = dash.Dash(__name__)


# Shared KPI style
kpi_style = {
    'flex': '1 1 10%',
    'margin': '8px',
    'textAlign': 'center',
    'color': 'white',
    'padding': '10px',
    'borderRadius': '8px',
    'minWidth': '100px',
    'backgroundColor': "#FFE9DE"
}

# -----------------------
# Layout (Responsive)
# -----------------------
app.layout = html.Div(
    [
        # -----------------------
        # Banner with Image + Title + Filters
        # -----------------------
        html.Div(
            [
                # Athlete picture
                html.Img(
                    src="assets/athlete.png",
                    style={
                        "height": "60px",
                        "width": "60px",
                        "border": "2px solid white",
                        "borderRadius": "50%",
                        "marginRight": "18px",
                        "marginLeft": "10px",
                        "flexShrink": 0
                    }
                ),

                # Title
                html.Div(
                    "Strava Dashboard",
                    style={
                        'color': 'white',
                        'fontSize': '28px',
                        'fontWeight': 'bold',
                        'textAlign': 'left',
                        'marginRight': '20px',
                        'flexGrow': 1
                    }
                ),

                # Filters
                html.Div([
                    # Date Range
                                        # Sport Type
                    html.Div([
                        dcc.Dropdown(
                            id='sport-type',
                            options=[{'label': s, 'value': s} for s in df['sport_type'].dropna().unique()],
                            value=None,
                            multi=True,
                            placeholder="Select sport type(s)",
                            style={'fontFamily':'Roboto, sans-serif', 'fontSize':'12px', 'minWidth':'150px', 'height':'28px'}
                        )
                    ], style={'display':'flex','flexDirection':'column'}),
                    html.Div([
                        dcc.DatePickerRange(
                            id='date-range',
                            min_date_allowed=df['start_date_local'].min().date(),
                            max_date_allowed=df['start_date_local'].max().date(),
                            start_date=df['start_date_local'].min().date(),
                            end_date=df['start_date_local'].max().date(),
                            style={'fontFamily':'Roboto, sans-serif', 'fontSize':'12px', 'height':'28px'}
                        )
                    ], style={'display':'flex','flexDirection':'column','marginRight':'10px'}),


                ], style={'display':'flex','alignItems':'center','flexWrap':'wrap','gap':'10px'})
            ],
            style={
                'display': 'flex',
                'alignItems': 'center',
                'backgroundColor': "#FC5400",
                'padding': '8px',
                'borderRadius': '8px',
                'marginBottom': '10px',
                'flexWrap': 'wrap',
                'gap': '15px',
                'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'  # subtle shadow
            }
        ),

        # -----------------------
        # Two-Column Main Dashboard
        # -----------------------
# -----------------------
# Two-Column Main Dashboard
# -----------------------
html.Div([
    # Left Column
    html.Div([
        # KPI Cards
        html.Div([
            html.Div([
                html.H3(id='total-distance', style={'fontSize':'16px','margin':'1px 0'}),
                html.H4("Total KM", style={'fontSize':'10px','margin':'1px 0'})
            ], style={**kpi_style, 'backgroundColor':'#FC5400', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)'}),

            html.Div([
                html.H3(id='total-elev', style={'fontSize':'16px','margin':'1px 0'}),
                html.H4("Total Elevation", style={'fontSize':'10px','margin':'1px 0'})
            ], style={**kpi_style, 'backgroundColor':'#FC5400', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)'}),

            html.Div([
                html.H3(id='avg-pace', style={'fontSize':'16px','margin':'1px 0'}),
                html.H4("Avg Pace", style={'fontSize':'10px','margin':'1px 0'})
            ], style={**kpi_style, 'backgroundColor':'#FC5400', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)'}),

            html.Div([
                html.H3(id='avg-speed', style={'fontSize':'16px','margin':'1px 0'}),
                html.H4("Avg Speed", style={'fontSize':'10px','margin':'1px 0'})
            ], style={**kpi_style, 'backgroundColor':'#FC5400', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)'}),

                        html.Div([
                html.H3(id='activity-count', style={'fontSize':'16px','margin':'1px 0'}),
                html.H4("Activities", style={'fontSize':'10px','margin':'1px 0'})
            ], style={**kpi_style, 'backgroundColor':'#FC5400', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)'}),

        ], style={'display':'flex', 'flexWrap':'wrap', 'justifyContent':'center', 'marginBottom':'10px'}),

        # Heatmap Card
        html.Div([
            html.H3("Workout Time Preference", style={'textAlign':'left','margin':'1px 0'}),
            dcc.Graph(id='heatmap', style={'width':'100%', 'height':'400px'})
        ], style={'backgroundColor':'#FFFFFF', 'padding':'10px', 'borderRadius':'10px', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)', 'marginBottom':'20px'}),

        # Personal Bests Table Card
        html.Div([
            html.H3("Personal Bests", style={'textAlign':'left','margin':'1px 0','margin-bottom':'10px'}),
            dash_table.DataTable(
    id='personal-bests-table',
    page_size=5,
    style_table={
        'width': '100%',          # table fills the container width
        'minWidth': '100%',       # ensures full width
        'overflowX': 'auto',      # allows horizontal scroll on small screens
        'height': 'auto',
        'maxHeight': '70vh',      # optional: table max height relative to viewport
    },
    style_header={
        'fontWeight': 'bold',
        'backgroundColor': '#f0f0f0',
        'whiteSpace': 'normal',   # wrap header text if needed
    },
    style_cell={
        'textAlign': 'center',
        'minWidth': '80px',       # minimum cell width
        'width': 'auto',          # allow cells to expand
        'maxWidth': '300px',      # prevent overly wide cells
        'whiteSpace': 'normal',   # wrap content text
    },
    page_action='native',         # enables pagination
    sort_action='native',         # allow sorting

)
        ], style={'backgroundColor':'#FFFFFF', 'padding':'10px', 'borderRadius':'10px', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)', 'marginBottom':'20px'}),

    ], style={'flex':'1 1 300px','minWidth':'300px'}),  # Left Column

    # Right Column
    html.Div([
        # Map Card
        html.Div([
            html.H3("Spatial Concentration & Route Map", style={'textAlign':'left','margin':'1px 0','margiBottom':'5px 0','margin-bottom':'10px'}),
            html.Iframe(
                id='folium-map',
                srcDoc=open('map.html','r').read() if os.path.exists('map.html') else "",
                width='100%',
                height='466',
                style={'border':'none'}
            )
        ], style={'backgroundColor':'#FFFFFF', 'padding':'10px', 'borderRadius':'10px', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)', 'marginBottom':'20px'}),

        # Weekly Distance Card
        html.Div([
            html.H3("Total Distance (km) by Week", style={'textAlign':'left','margin': '1px 0px'}),
            dcc.Graph(id='weekly-distance', style={'width':'100%', 'height':'230px'})
        ], style={'backgroundColor':'#FFFFFF', 'padding':'10px', 'borderRadius':'10px', 'boxShadow':'0 4px 8px rgba(0,0,0,0.15)'}),

    ], style={'flex':'1 1 700px','minWidth':'300px'}),  # Right Column

], style={'display':'flex','flexWrap':'wrap','gap':'12px','marginBottom':'40px'})

    ],
    style={
        'fontFamily':'Roboto, sans-serif',
        'backgroundColor':'#f9f9f9',
        'color':'#333',
        'margin':'18px'
    }
)



# -----------------------
# Callbacks
# -----------------------
@app.callback(
    Output('total-distance','children'),
    Output('avg-pace','children'),
    Output('total-elev','children'),
    Output('activity-count','children'),
    Output('avg-speed','children'),
    Output('heatmap','figure'),
    Output('folium-map','srcDoc'),
    Output('weekly-distance','figure'),
    Output('personal-bests-table','columns'),
    Output('personal-bests-table','data'),
    Input('date-range','start_date'),
    Input('date-range','end_date'),
    Input('sport-type','value')
)
def update_dashboard(start_date, end_date, sport_types):
    dff = df[(df['start_date_local'].dt.date >= pd.to_datetime(start_date).date()) &
             (df['start_date_local'].dt.date <= pd.to_datetime(end_date).date())]
    if sport_types:
        dff = dff[dff['sport_type'].isin(sport_types)]

    total_distance, avg_pace_str, total_elev, activity_count, avg_speed = compute_metrics(dff)
    heatmap_fig = create_heatmap_figure(dff)
    map_file = create_folium_map(dff)
    with open(map_file,'r') as f:
        map_src = f.read()
    weekly_fig = weekly_distance_figure(dff)
    columns, data = personal_bests_table(dff)

    return total_distance, avg_pace_str, total_elev, activity_count, avg_speed, heatmap_fig, map_src, weekly_fig, columns, data

# -----------------------
# Run App
# -----------------------
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

### The app is running on http://127.0.0.1:8050/ (open a browser and go to this address)